# Нейронные сети. Архитектуры нейронных сетей

## Подготовка данных

В этом практическом заданий мы будем решать задачу классификации цифр на датасете `mnist` с помощью полносвязной и сверточной нейронной сети. Для этого мы будем использовать надстройку над `tensorflow`, которая называется `keras`. Для начала обсудим данные. `mnist` датасет состоит из черно-белых изображений цифр размером $28 \times 28$ пикселей. В данном случае, мы работаем с одним каналом, хотя в случае цветных изображений, общее число каналов равно трем. Загрузим наши данные используя функцию `load_data` объекта `mnist` из модуля `keras.dataset`. Перед выполнением этого задания убедитесь, что ваша версия `tensorflow` >= 1.4.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd

In [3]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [4]:
tf.__version__

'2.2.0'

Нормализуйте заруженные данные `x_train` и `x_test`. Для этого следует разделить числовое значение каждого пикселя на $255$. Далее, переведите `y_train` и `y_test` в one-hot представление, используя функцию `tf.keras.utils.to_categorical`. Наше первое задание будет заключатся в реализации полносвязной нейронной сети. Поэтому измените размерность тренировочных и тестовых данных с помощью метода `reshape`.
    >> np_vector.shape
    >> (28, 28)
    >> np_vector = np_vector.reshape(28 * 28)
    >> np_vector.shape
    >> (784,)

### *РЕШЕНИЕ*

In [5]:
# нормализация
x_train = x_train/255
x_test = x_test/255

In [6]:
y_train = tf.keras.utils.to_categorical(y_train,10)
y_test = tf.keras.utils.to_categorical(y_test,10)

In [7]:
x_train.shape

(60000, 28, 28)

In [8]:
x_train = x_train.reshape(60000, 28*28)

In [9]:
x_train.shape

(60000, 784)

In [10]:
x_test.shape

(10000, 28, 28)

In [11]:
x_test = x_test.reshape(10000, 28*28)

In [12]:
x_test.shape

(10000, 784)

## Полносвязная нейронная сеть.

В этой части задания вам предлагается реализовать обычную нейронную сеть с использованием последовательной модели `tf.keras.models.Sequential`. Данная модель позволяет добавлять слои при помощи функции встроенной `add`. Наша нейронная сеть будет состоять всего лишь из одного скрытого слоя с количеством нейроннов равным $256$, функцией активации ReLU и с `input_shape=(784,)`, что соответствует количеству нейронов во входном слое нашей нейронной сети. Количество нейроннов в выходном слое равно количеству классов, в качестве функции активации нужно использовать softmax. Не забудьте вызвать `model.compile` после добавления слоев. Используйте в качестве функции потерь `categorical_crossentropy`, оптимизатор `adadelta` и метрику `accuracy`.

### *РЕШЕНИЕ*

In [13]:
model1 = tf.keras.models.Sequential()
model1.add(tf.keras.layers.Dense(256, activation='relu', input_shape=(784,)))
model1.add(tf.keras.layers.Dropout(0.2))
model1.add(tf.keras.layers.Dense(10, activation='softmax'))

In [14]:
model1.compile(
            optimizer='adadelta',
            loss='categorical_crossentropy',
            metrics='accuracy')

После этого, создайте модель и загрузите веса нейронной сети из файла `neural_networks.h5`. Какое количество настраиваемых параметров содержится в этой нейронной сети. Запишите это число в качестве первого ответа `answer1` на это задание. Оцените качетво на тестовой выборке и запишите это значение с точностью до трех знаков после запятой в переменную `answer2`.

### *РЕШЕНИЕ*

In [15]:
model1.load_weights('neural_networks.h5')

In [16]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               200960    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
_________________________________________________________________


In [17]:
answer1=203530

In [18]:
_ = model1.fit(x_train, y_train)

1875/1875 [==============================] - 3s 2ms/step - loss: 0.0428 - accuracy: 0.9862


In [19]:
loss, accuracy = model1.evaluate(x_test, y_test)

313/313 [==============================] - 0s 972us/step - loss: 0.0656 - accuracy: 0.9823


In [20]:
print('loss {}, accuracy: {}'.format(loss,accuracy))

loss 0.06562840938568115, accuracy: 0.9822999835014343


In [21]:
answer2=round(accuracy,3)
answer2

0.982

## Сверточная нейронная сеть

Далее, вам предлагается реализовать сверточную нейронную сеть. 

* Размерность входного слоя $(28, 28, 1)$.
* Сверточный слой с $32$ каналами, ядро свертки $3 \times 3$.
* Макспулинг слой $(2,2)$.
* Сверточный слой с $64$ каналами, ядро свертки $3 \times 3$.
* Макспулинг слой $(2,2)$.
* Понижение размерности признаков.
* Полносвязный слой с 64 нейронами
* Выходной слой с количеством нейронов равному количеству классов.

Для этого предлагается использовать следующие классы `Convolution2D`, `MaxPooling2D` и `Flatten` для понижения размерности признаков. Все эти классы как и слой полносвязной нейронной сети `Dense` находятся в `tf.keras.layers`. Используйте ReLU в качестве функции активации во всех слоях, где это потребуется, кроме выходного, в нем по аналогии с прошлым заданием используется softmax. Аналогичная ситуация с функцией `compile` после добавления слоев.

### *РЕШЕНИЕ*

In [22]:
model2 = tf.keras.models.Sequential()

model2.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu',
                       input_shape=(28,28,1)))
           
model2.add(tf.keras.layers.MaxPooling2D((2,2)))
           
model2.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model2.add(tf.keras.layers.MaxPooling2D((2,2)))
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(64, activation='relu'))
model2.add(tf.keras.layers.Dense(10, activation='softmax'))           

In [23]:
model2.compile(optimizer='adadelta',
            loss='categorical_crossentropy',
            metrics='accuracy')


Теперь оцените качество получившейся модели на тестовой выборке. Для этого измените размерность `x_train` и `x_test` на размерность входного слоя. Загрузите веса `conv_net.h5`. Запишите количество параметров этой сверточной нейронной сети в `answer3`. Сравните его с количеством параметром в полносвязной нейронной сети, которую мы реализовали ранее. Оценку качества запишите в `answer4` с точностью до 3 трех знаков после запятой.

### *РЕШЕНИЕ*

In [24]:
x_train.shape

(60000, 784)

In [25]:
x_train = x_train.reshape((60000, 28, 28,1))

In [26]:
x_test.shape

(10000, 784)

In [27]:
x_test = x_test.reshape((10000, 28, 28,1))

In [28]:
model2.load_weights('conv_net.h5')

In [29]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                102464    
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

In [30]:
answer3=121930

In [31]:
_ = model2.fit(x_train, y_train)

1875/1875 [==============================] - 20s 10ms/step - loss: 0.0031 - accuracy: 0.9993


In [32]:
loss2, accuracy2 = model2.evaluate(x_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.0303 - accuracy: 0.9929


In [33]:
print('loss {}, accuracy: {}'.format(loss2,accuracy2))

loss 0.030259884893894196, accuracy: 0.992900013923645


In [34]:
answer4=round(accuracy2,3)

# Строка с ответами

In [35]:
output = "nn params {0}\n nn score {1:.3f}\ncnn params {2}\ncnn score {3:.3f}"
print(output.format(answer1, answer2, answer3, answer4))

nn params 203530
 nn score 0.982
cnn params 121930
cnn score 0.993
